In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import torch
import tqdm
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
import logging

Set up the U-Net model

In [2]:
#This is the U-Net architecture
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
        
    def forward(self, x):
        return self.double_conv(x)

class Down(nn.Module):
    """Downscaling with maxpool then double conv"""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""
    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()
        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_channels // 2, in_channels // 2, kernel_size=2, stride=2)
        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = torch.tensor([x2.size()[2] - x1.size()[2]])
        diffX = torch.tensor([x2.size()[3] - x1.size()[3]])

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 512)
        self.up1 = Up(1024, 256, bilinear)
        self.up2 = Up(512, 128, bilinear)
        self.up3 = Up(256, 64, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

Load the prepared Data

In [4]:
data=[np.load("./herm-ims.npy"),np.load("./herm-tags.npy")]
dataset = torch.utils.data.TensorDataset(torch.Tensor(data[0]).unsqueeze(1),torch.Tensor(data[1]).unsqueeze(1).type(torch.int64))

Split train, validation (2537 and 400 here)

In [7]:
train,val=torch.utils.data.random_split(dataset,[2537,len(dataset)-2537])

In [12]:
batch_size=1
traindataloader = torch.utils.data.DataLoader(train,batch_size=batch_size,shuffle=True,num_workers=2)
valdataloader = torch.utils.data.DataLoader(val,batch_size=batch_size,shuffle=True,num_workers=2)
n_train=len(train)
n_val=len(val)
print("Training Samples:",n_train)
print("Validation Samples:",n_val)

Training Samples: 2537
Validation Samples: 400


In [13]:
lr=0.001
net = UNet(n_channels=1, n_classes=1)
optimizer = torch.optim.RMSprop(net.parameters(), lr=lr, weight_decay=1e-8)
criterion = nn.BCEWithLogitsLoss()
#The following is to bias the training
#pos_weight = torch.full((1,290,190),5.)
#criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
writer = SummaryWriter(comment=f'LR_{lr}_BS_{batch_size}')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs=20
net.to(device=device)
print("",end="")

In [14]:
#Dice coefficient for accuracy
from torch.autograd import Function
class DiceCoeff(Function):
    """Dice coeff for individual examples"""

    def forward(self, input, target):
        self.save_for_backward(input, target)
        eps = 0.0001
        self.inter = torch.dot(input.view(-1), target.view(-1))
        self.union = torch.sum(input) + torch.sum(target) + eps

        t = (2 * self.inter.float() + eps) / self.union.float()
        return t

    # This function has only a single output, so it gets only one gradient
    def backward(self, grad_output):

        input, target = self.saved_variables
        grad_input = grad_target = None

        if self.needs_input_grad[0]:
            grad_input = grad_output * 2 * (target * self.union - self.inter) \
                         / (self.union * self.union)
        if self.needs_input_grad[1]:
            grad_target = None

        return grad_input, grad_target


def dice_coeff(input, target):
    """Dice coeff for batches"""
    s = torch.FloatTensor(1).zero_()
    for i, c in enumerate(zip(input, target)):
        s = s + DiceCoeff().forward(c[0], c[1])
    return s / (i + 1)

In [15]:
def eval_net(net, loader, device, n_val,writer,global_step):
    """Evaluation without the densecrf with the dice coefficient"""
    net.eval()
    tot = 0

    with tqdm.tqdm(total=n_val, desc='Validation round', unit='img', leave=False) as pbar:
        for batch in loader:
            imgs = batch[0]
            #print(imgs.shape)
            labels = batch[1]
            assert imgs.shape[1] == net.n_channels, \
                f'Network has been defined with {net.n_channels} input channels, ' \
                f'but loaded images have {imgs.shape[1]} channels. Please check that ' \
                'the images are loaded correctly.'

            imgs = imgs.to(device=device, dtype=torch.float32)
            label_type = torch.float32
            labels= labels.to(device=device, dtype=label_type)

            preds = net(imgs)

            writer.add_images('images', imgs, global_step)
            writer.add_images('masks/true', labels, global_step)
            writer.add_images('masks/pred', torch.sigmoid(preds) > 0.5, global_step)          
            
            for label, pred in zip(labels, preds):
                pred = (pred > 0.5).float()
                tot += dice_coeff(pred, label.squeeze(dim=1)).item()
            pbar.update(imgs.shape[0])

    return tot / n_val

In [ ]:
global_step = 0
for epoch in range(num_epochs):
    
    epoch_loss = 0
    with tqdm.tqdm(total=n_train, desc=f'Epoch {epoch + 1}/{num_epochs}', unit='img') as pbar:
        
        for batch in traindataloader:
            net.train()
            imgs = batch[0]
            #print(imgs.shape)
            labels = batch[1]
            assert imgs.shape[1] == net.n_channels, \
                f'Network has been defined with {net.n_channels} input channels, ' \
                f'but loaded images have {imgs.shape[1]} channels. Please check that ' \
                'the images are loaded correctly.'

            imgs = imgs.to(device=device, dtype=torch.float32)
            label_type = torch.float32
            labels= labels.to(device=device, dtype=label_type)

            preds = net(imgs)
            #print(preds.shape)
            #print(labels.shape)
            loss = criterion(preds,labels)
            epoch_loss += loss.item()
            writer.add_scalar('Loss/train', loss.item(), global_step)

            pbar.set_postfix(**{'loss (batch)': loss.item()})

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            pbar.update(imgs.shape[0])
            global_step += 1
            if global_step % (len(dataset) // (10 * batch_size)) == 0:
                val_score = eval_net(net, valdataloader, device, n_val,writer,global_step)
                writer.add_scalar('Dice/test', val_score, global_step)
                writer.add_images('images', imgs, global_step)
                writer.add_images('masks/true', labels, global_step)
                writer.add_images('masks/pred', torch.sigmoid(preds) > 0.5, global_step)

writer.close()

Validation round:  56%|█████▋    | 226/400 [03:46<02:53,  1.01img/s]


Validation round:  13%|█▎        | 51/400 [00:51<05:45,  1.01img/s]


Validation round:  70%|██████▉   | 278/400 [04:36<02:01,  1.00img/s]


Validation round:  26%|██▌       | 104/400 [01:43<04:53,  1.01img/s]


Validation round:  82%|████████▎ | 330/400 [05:41<01:16,  1.09s/img]


...

Validation round:  90%|████████▉ | 358/400 [05:56<00:41,  1.02img/s]


Validation round:  46%|████▌     | 184/400 [03:05<03:34,  1.01img/s]


Validation round:   2%|▏         | 8/400 [00:08<06:43,  1.03s/img]


Validation round:  59%|█████▉    | 236/400 [03:55<02:41,  1.02img/s]


Epoch 3/20:  71%|███████   | 1792/2537 [2:17:33<49:16,  3.97s/img, loss (batch)=0.0415]    

Save the network

In [17]:
#torch.save(net.state_dict(), './Segnet.pth')

We can see the real time plots running tensorboard in the directory "runs"